# Basics of Pandas
Pandas is a great tool for manipulating tabular data in Python, which makes it a great way to learn basic functions for working with datasets.

We'll create an example where we compare the experimental and computed bandgaps for different materials.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd

Data locations. We are going to use [experiment](https://foundry-ml.org/#/datasets/10.18126%2Fwg3u-g8vu) and [computational](https://foundry-ml.org/#/datasets/10.18126%2F7io9-1z9k) datasets hosted by the [Foundry project](https://foundry-ml.org/#/).

In [ ]:
exp_url = 'https://data.materialsdatafacility.org/foundry/foundry_experimental_band_gaps_v1.1/data_exp_no_repeat_new.json'
dft_url = 'https://data.materialsdatafacility.org/foundry/foundry_assorted_computational_band_gaps_v1.1/All_no_MP_Aflow_OQMD_new.json'

## Reading
Pandas [can read from numerous tabular data formats](https://pandas.pydata.org/docs/user_guide/io.html). All are all named `read_X` and can download data from the internet for you.

In [ ]:
exp_data = pd.read_json(exp_url, lines=True)
print(f'Loaded {len(exp_data)} records')

It's that easy. We now have a tabular dataset with a few thousand records and more data than we'll need, such as the full structure.

In [ ]:
exp_data.head()

You can delete columns and from the dataset pretty easily.

In [ ]:
exp_data.drop(columns=['structure'], inplace=True)  # Inplace alters the variable. Pandas returns a new copy by default, leaving the original intact
exp_data.head()

## Cleaning the data
The dataset is a little messy. Some of the columns have lists for bandgap values rather than single values and those single values are strings and not floats, which complicate analysis but yield an teaching moment for the benefits of Pandas.

Let's show how to remove them and process the data into useful values

In [ ]:
try:
    exp_data['bandgap value (eV)'].max()
except TypeError as e:
    print(e)

Pandas has a [built-in function for converting strings to numbers](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html)

In [ ]:
pd.to_numeric(exp_data['bandgap value (eV)'], errors='coerce')

It returns a list of floats, which is exactly what we need. So, let's store that value as the new column because we don't need the old one

In [ ]:
exp_data['bandgap value (eV)'] = pd.to_numeric(exp_data['bandgap value (eV)'], errors='coerce')

You may have noticed some of th evalues are not numbers (`NaN`s). We can use Pandas's selection tools for removing them.

One route is to supply a list of booleans.

In [ ]:
exp_data['bandgap value (eV)'].isnull()

In [ ]:
exp_data = exp_data[~exp_data['bandgap value (eV)'].isnull()]
print(f'Reduced to {len(exp_data)} values')

You can also query. For example, to get the only the bandgaps which we know to be direct.

In [ ]:
exp_data.query('`bandgap type`=="D"')  # `` are used because the column name has spaces

The full query syntax supports all kind of complex expressions

## Aggregating
Many common data analysis require running the same class of analysis on different subsets of the data, "aggregation" operations are key for this. 

There are many [aggregation operations in Pandas](https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html). Simple ones like getting descriptive statistics

In [ ]:
exp_data['bandgap value (eV)'].mean()

In [ ]:
exp_data['bandgap value (eV)'].agg(['mean', 'count', 'sem', 'std'])

In [ ]:
exp_data['bandgap value (eV)'].describe()

Or complicated ones like counting numbers of occurances

In [ ]:
exp_data['composition'].value_counts()

They can also be performed on specific groups of data. Form groups by using the "group by" operation, 

In [ ]:
exp_data.groupby('composition')['bandgap value (eV)'].agg(['mean', 'max', 'std', 'sem'])

Grouping can even take multiple tags and be chained together with query operations

> Chaining is why `inplace=False` is a useful default in Pandas

In [ ]:
exp_data.query('`bandgap type` != "D"').groupby(['composition', 'space group'])['bandgap value (eV)'].mean()

Let's end by making a single dataset with the average bandgap for each composition and spacegroup

In [ ]:
exp_bandgaps = exp_data.groupby(['composition', 'space group'])['bandgap value (eV)'].mean().reset_index()  # reset turns the single-column series into a dataframe
exp_bandgaps

We'll give you some exercises later to help you see the power of aggregation operations.

## Joining
Data analyses often require _joining_ two different sources of data together. We'll showcase this by analyzing the differences between experiment and computation.

In [ ]:
dft_data = pd.read_json(dft_url, lines=True).drop(columns=['structure', 'icsd_id'])
dft_data.head()

In [ ]:
dft_data['comp method'] = dft_data['comp method'].str.upper()

There are [many types of join operations](https://www.devart.com/dbforge/sql/sqlcomplete/sql-join-statements.html). [Prepare for Venn Diagrams](https://www.devart.com/dbforge/sql/sqlcomplete/sql-join-statements.html) if you want to learn more about them.

A common type are "inner joins" where you get the rows from different datasets with values that match

In [ ]:
joined_data = dft_data.merge(exp_data, on=['composition', 'space group'], suffixes=('_dft', '_exp'))
joined_data.head()

Note how we know have band gap values for both experiment and each computational record

In [ ]:
joined_data['error'] = joined_data['bandgap value (eV)_exp'] - joined_data['bandgap value (eV)_dft']

In [ ]:
joined_data.groupby('comp method')['error'].describe()

Plotting to make a visual display. Pandas does have [some plotting helpers](https://pandas.pydata.org/docs/user_guide/visualization.html)

In [ ]:
num_methods = joined_data['comp method'].value_counts().size
print(f'Found {num_methods} computational methods')

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(8.5, 2.), sharey=True, sharex=True)

for ax, (method, group) in zip(axs, joined_data.groupby('comp method')):
    ax.scatter(group['bandgap value (eV)_dft'], group['bandgap value (eV)_exp'], s=1)
    ax.text(0.01, 0.99, method, transform=ax.transAxes, ha='left', va='top')
    ax.set_xlabel(f'$E_g$, {method} (eV)')

# Get the range
max_eg = max(max(ax.get_xlim()) for ax in axs)
ax.set_xlim(0, max_eg)
ax.set_ylim(ax.get_xlim())

# Make a y=x line
for ax in axs:
    ax.plot(ax.get_xlim(), ax.get_xlim(), 'k--', zorder=-1)
axs[0].set_ylabel('$E_g$, Exp (eV)')
    
fig.tight_layout()

Pandas makes it easy to manipulate data.

## Exercises
Here are a few questions which will test your ability to use Pandas. Try to do them in as few steps as possible.

_Add a column to `exp_data` which is the temperature in Celcius_.
<code hidden>exp_data['temp (C)'] = pd.to_numeric(exp_data['temp (K)'], errors='coerce') - 273.15</code>

_Which composition, space group, and method have the largest error?_
<code hidden>joined_data.loc[joined_data['error'].abs().idxmax(), ['composition', 'space group', 'comp method', 'error']]</code>

_How many compositions have experiment bandgaps at more than one temperature_?
<code hidden>(exp_data.groupby('composition')['temp (K)'].value_counts() > 1).sum()</code>

_What is the median error between each method and experiment?_
<code hidden>joined_data.groupby('comp method')['error'].median()</code>

_What is the mean absolute error after subtracting off this median error_? Hint: I used [apply](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html).
<code hidden>joined_data.groupby('comp method').apply(lambda x: (x['error'] - x['error'].median()).abs().mean())</code>

_Which are the compounds where 4 of 5 computational methods have been evaluated_?
<code hidden>joined_data.drop_duplicates(['composition', 'space group', 'comp method']).groupby(['composition', 'space group']).filter(lambda x: len(x) == 4)['composition'].value_counts()</code>